In [1]:
% matplotlib inline
%config InlineBackend.figure_format = 'retina'

from __future__ import division

import numpy as np
import glob
import matplotlib.pyplot as plt
import scipy.sparse as sps
import scipy.linalg as sl


import enterprise
from enterprise.pulsar import Pulsar
from enterprise.signals import parameter
from enterprise.signals import white_signals
from enterprise.signals import utils
from enterprise.signals import gp_signals
from enterprise.signals import signal_base
from enterprise.signals import selections

import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

In [2]:
# compute f_p statistic
def fpStat(psr, f0):
    """ 
    Computes the Fp-statistic as defined in Ellis, Siemens, Creighton (2012)
    
    :param psr: List of pulsar object instances
    :param f0: Gravitational wave frequency
    :return: Value of the Fp statistic evaluated at f0
    """

    fstat=0.
    npsr = len(psr)

    # define N vectors from Ellis et al, 2012 N_i=(x|A_i) for each pulsar
    N = np.zeros(2)
    # define M matrix M_ij=(A_i|A_j)
    M = np.zeros((2, 2))
    for ii,p in enumerate(psr):

        # Define A vector
        A = np.zeros((2, len(p.toas)))
        A[0,:] = 1./f0**(1./3.) * np.sin(2*np.pi*f0*p.toas)
        A[1,:] = 1./f0**(1./3.) * np.cos(2*np.pi*f0*p.toas)

        # define N vectors from Ellis et al, 2012 N_i=(x|A_i) for each pulsar
        N = np.array([np.dot(A[0,:], np.dot(p.invCov, p.residuals)), \
                      np.dot(A[1,:], np.dot(p.invCov, p.residuals))]) 
        
        # define M matrix M_ij=(A_i|A_j)
        for jj in range(2):
            for kk in range(2):
                M[jj,kk] = np.dot(A[jj,:], np.dot(p.invCov, A[kk,:]))
        
        # take inverse of M
        Minv = np.linalg.inv(M)
        fstat += 0.5 * np.dot(N, np.dot(Minv, N))

    # return F-statistic
    return fstat

In [3]:
# compute f_e-statistic
def feStat(psr, gwtheta, gwphi, f0):
    """ 
    Computes the F-statistic as defined in Ellis, Siemens, Creighton (2012)
    
    :param psr: List of pulsar object instances
    :param gwtheta: GW polar angle
    :param gwphi: GW azimuthal angle
    :param f0: Gravitational wave frequency
    :return: Value of the Fe statistic evaluated at gwtheta, phi, f0
    """
    
    npsr = len(psr)
    N = np.zeros(4)
    M = np.zeros((4,4))
    for ii, p in enumerate(psr):
        fplus, fcross, cosMu = createAntennaPatternFuncs(p, gwtheta, gwphi)

        # define A
        A = np.zeros((4, len(p.toas)))
        A[0,:] = fplus/f0**(1./3.) * np.sin(2*np.pi*f0*p.toas)
        A[1,:] = fplus/f0**(1./3.) * np.cos(2*np.pi*f0*p.toas)
        A[2,:] = fcross/f0**(1./3.) * np.sin(2*np.pi*f0*p.toas)
        A[3,:] = fcross/f0**(1./3.) * np.cos(2*np.pi*f0*p.toas)


        N += np.array([np.dot(A[0,:], np.dot(p.invCov, p.res)), \
                        np.dot(A[1,:], np.dot(p.invCov, p.res)), \
                        np.dot(A[2,:], np.dot(p.invCov, p.res)), \
                        np.dot(A[3,:], np.dot(p.invCov, p.res))]) 

        M += np.dot(A, np.dot(p.invCov, A.T))

    # inverse of M
    Minv = np.linalg.pinv(M)

    # Fe-statistic
    return 0.5 * np.dot(N, np.dot(Minv, N))

In [4]:
def invCov(psr):
    

SyntaxError: unexpected EOF while parsing (<ipython-input-4-551579a0ff0d>, line 2)

In [2]:
datadir = enterprise.__path__[0] + '/datafiles/ng9/'
#datadir = '~/Documents/Grad\ School/Research/enterprise/enterprise/datafiles'
parfiles = sorted(glob.glob(datadir + '/*.par'))
timfiles = sorted(glob.glob(datadir + '/*.tim'))

In [3]:
#Trial with a few pulsars
parfile_J0030 = datadir + 'J0030+0451_NANOGrav_9yv1.gls.par'
parfile_B1937 = datadir + 'B1937+21_NANOGrav_9yv1.gls.par'


timfile_J0030 = datadir + 'J0030+0451_NANOGrav_9yv1.tim'
timfile_B1937 = datadir + 'B1937+21_NANOGrav_9yv1.tim'

noisefile_J0030 = datadir + 'J0030+0451_noise.txt'
noisefile_B1937 = datadir + 'B1937+21_noise.txt'

psr_J0030 = Pulsar(parfile_J0030,timfile_J0030)
psr_B1937 = Pulsar(parfile_B1937,timfile_B1937)

pulsars = [psr_J0030,psr_B1937]

In [4]:
##### parameters and priors #####

# white noise parameters
# since we are fixing these to values from the noise file we set
# them as constant parameters
efac = parameter.Uniform(0.1,5.0)
equad = parameter.Uniform(-10.0,-5.0)
ecorr = parameter.Constant()

'''
# red noise parameters
log10_A = parameter.LinearExp(-20,-12)
gamma = parameter.Uniform(0,7)

# GW parameters (initialize with names here to use parameters in common across pulsars)
log10_A_gw = parameter.LinearExp(-18,-12)('log10_A_gw')
gamma_gw = parameter.Constant(4.33)('gamma_gw')
'''

# timing model
tm = gp_signals.TimingModel()

##### Set up signals #####

# white noise
ef = white_signals.MeasurementNoise(efac=efac)
eq = white_signals.EquadNoise(log10_equad=equad)
#ec = white_signals.EcorrKernelNoise(log10_ecorr=ecorr, selection=selection)

# full model is sum of components
model = tm + ef# + eq

# intialize PTA
pta_one = signal_base.PTA(model(psr_J0030))

pta_test = signal_base.PTA([model(pulsar) for pulsar in pulsars])

In [5]:
#Look at parameters and put in dictionary definition
xs = {par.name: par.sample() for par in pta_test.params}
print(xs)

{'B1937+21_efac': 3.097294598109011, 'J0030+0451_efac': 0.6535716222855898}


In [9]:
params = xs if isinstance(xs,dict) else pta_test.map_params(xs)

# phiinvs will be a list or may be a big matrix if spatially
# correlated signals
TNrs = pta_test.get_TNr(params)
TNTs = pta_test.get_TNT(params)
phiinvs = pta_test.get_phiinv(params, logdet=True, method='partition')

# red noise piece
if pta_test._commonsignals:
    print('red')
    phiinv, logdet_phi = phiinvs

    Sigma = pta_test._make_sigma(TNTs, phiinv)
    cf = signal_base.cholesky(Sigma)
else:
    #takes individual block matrices (ie TNT) and factorizes their sigmas instead of using sparse cholesky like above
    for TNT, (phiinv, logdet_phi) in zip(TNTs, phiinvs):
        Sigma = TNT + (np.diag(phiinv) if phiinv.ndim == 1 else phiinv)
        cf = sl.cho_factor(Sigma)

#inverts sigma
SigmaInv = sl.cho_solve(cf,np.eye(len(cf[0])))


(2, 2)
(2,)


In [7]:
#pta_one.summary()
#pta_test_lnlike = signal_base.MarginalizedLogLikelihood(pta_test)
TNTs = pta_test.get_TNT(xs)
TNrs = pta_test.get_TNr(xs)
phiinv = pta_test.get_phiinv(xs, logdet=True,method='partition')
print(np.shape(phiinv))
print(np.shape(TNTs))
pta_one_lnlike._make_sigma(TNTs,phiinv)

(2, 2)
(2,)


NameError: name 'pta_one_lnlike' is not defined

In [13]:
#All of the Pulsars!
psrs = []
for p, t in zip(parfiles, timfiles):
    psr = Pulsar(p, t)
    psrs.append(psr)

In [14]:
'''
print(psrs[0].residuals)
print(len(psrs[0].toas))
'''

'\nprint(psrs[0].residuals)\nprint(len(psrs[0].toas))\n'

In [15]:
f_gw = 8e-8
fp = fpStat(psrs,f_gw)

AttributeError: 'Tempo2Pulsar' object has no attribute 'invCov'